<center>
<h1 style="color:red; font-size:70px;"> Esercitazione 8</h1>

In [2]:
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import layers, activations

from keras.layers import Input, Flatten, Dense, Rescaling, MaxPooling2D, Conv2D, Dropout

<h1 style="color: blue; font-size:50px;"> Esercizio 1 - Data augmentation</h1>

L'obiettivo di questo esercizio è vedere in azione la tecnica del "data augmentation", che consente di aumentare artificialmente la quantità di dati disponibili per l'addestramento di un modello, senza che debbano essere raccolti nuovi dati reali. Questo consente di migliorare la performance del modello ed allo stesso tempo di ridurre il rischio di overfitting (in quanto è più difficile che il modello impari troppo bene i dati d'addestramento). Nel campo delle immagini solitamente per aumentare la dimensione del campione si fa uso di:

- rotazioni

- riflessioni

- zoom o ritagli

- variazioni di luminosità o contrasto

- traslazioni

- aggiunta di rumore

In [3]:
# Come primo step per la risoluzione dell'esercizio importo il dataset sul quale vogliamo allenare una 
# rete neurale pensata per risolvere un problema di classificazione. Nello specifico sono 3670 immagini 
# di fiori, ciascuna delle quali è un tensore tridimensionale di (180, 180, 3) pixel
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
